Сделать и улучшить любую ML-модель на ваших проектных данных (просто клф, бленд, ранжирование, что-то что вы придумали сами...), используя любые признаки, какие захотите. Оцениваться будут:

факт выполнения задания :)
корректность кода (чтобы код не падал) и отсутствие логических ошибок (e.g. затестили на трейне)
итеративность улучшения (например взяли один сет признаков, показали качество; потом добавили / подкрутили / использовали другую модель, показали качество...)
креативность признаков
аккуратность ноутбука

In [ ]:
!pip install razdel
!pip install pymorphy2

import pandas as pd
import numpy as np
from razdel import tokenize
from razdel import sentenize
import nltk
from nltk.corpus import stopwords
from string import punctuation
import pymorphy2

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.svm import SVC


morph = pymorphy2.MorphAnalyzer()
nltk.download('stopwords')  
nltk.download('punkt')
russian_stopwords = stopwords.words('russian')

queries = pd.read_excel('/content/queries.xlsx')
answers = pd.read_excel('/content/answers.xlsx')

In [ ]:
from sklearn.model_selection import train_test_split

columns = ['Номер связки', 'deepmint_preprocessed', 'natasha_preprocessed']

answers = pd.DataFrame(answers, columns=columns)
queries = pd.DataFrame(queries, columns=columns)

data = pd.concat([answers, train])


In [32]:
vectorizer = TfidfVectorizer()

def scores(model):
    print('Accuracy: ', round(model.score(X_test, y_test),4))
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average = 'weighted')
    print(f'F1 score: {round(f1,4)}')

    return '\n'


def logistic_regression():
    model = LogisticRegression()
    model.fit(X_train, y_train)

    return model

def GBC():
    model = GradientBoostingClassifier(criterion='mse')
    model.fit(X_train, y_train)

    return model
  
def gaussian_NB():
    model = GaussianNB()
    model.fit(X_train, y_train)

    return model

def multinominal_NB():
    model = MultinomialNB()
    model.fit(X_train, y_train)

    return model


def bernulli_NB():
    model = BernoulliNB()
    model.fit(X_train, y_train)

    return model

def svc():
    model = SVC(kernel='linear')
    model.fit(X_train, y_train)

    return model




if __name__ == "__main__":

  
    # PLAIN
    vec = vectorizer.fit_transform(data['deepmint_preprocessed'].values.astype('U'))
    X = vec.toarray()

    data = data.fillna(0)
    data.astype({'Номер связки': 'int32'}).dtypes
    y = data['Номер связки']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23434)


    print(scores(gaussian_NB()))
    print(scores(multinominal_NB()))
    print(scores(bernulli_NB ()))
    print(scores(logistic_regression()))
    print(scores(GBC()))
    print(scores(svc()))

Accuracy:  0.4484
F1 score: 0.4152


Accuracy:  0.3835
F1 score: 0.214


Accuracy:  0.4071
F1 score: 0.2698


Accuracy:  0.5103
F1 score: 0.4111


Accuracy:  0.5457
F1 score: 0.5103


Accuracy:  0.6165
F1 score: 0.5545




In [ ]:
# Метод опорных векторов показал лучший результат (как accuracy, так и F1)
# Попробуем оптимизировать параметры с помощью гридсерча

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        SVC(), tuned_parameters, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted sample

Best parameters set found on development set:

{'C': 10, 'kernel': 'linear'}

Grid scores on development set:

0.012 (+/-0.001) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.012 (+/-0.001) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.012 (+/-0.001) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.012 (+/-0.001) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.073 (+/-0.028) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.012 (+/-0.001) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.429 (+/-0.079) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.073 (+/-0.028) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.275 (+/-0.110) for {'C': 1, 'kernel': 'linear'}
0.434 (+/-0.090) for {'C': 10, 'kernel': 'linear'}
0.412 (+/-0.076) for {'C': 100, 'kernel': 'linear'}
0.412 (+/-0.076) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00         1
         1.0       0.38      0.58      0.46        24
         3.0       0.50      0.25      0.33         4
         5.0       1.00      0.40      0.57         5
         6.0       0.50      0.49      0.49        37
        10.0       0.00      0.00      0.00         2
        12.0       0.54      0.39      0.45        18
        21.0       1.00      0.50      0.67         2
        32.0       0.00      0.00      0.00         2
        37.0       0.32      0.38      0.35        26
        38.0       0.00      0.00      0.00         4
        43.0       0.00      0.00      0.00         0
        45.0       0.00      0.00      0.00         4
        46.0       1.00      1.00      1.00         1
        56.0       1.00      1.00      1.00         1
        57.0       0.00      0.00      0.00         3
        70.0       0.60      0.60      0.60         5
        74.0       1.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and 

Best parameters set found on development set:

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.031 (+/-0.003) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.031 (+/-0.003) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.031 (+/-0.003) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.031 (+/-0.003) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.037 (+/-0.002) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.031 (+/-0.003) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.381 (+/-0.088) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.037 (+/-0.002) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}
0.242 (+/-0.100) for {'C': 1, 'kernel': 'linear'}
0.379 (+/-0.092) for {'C': 10, 'kernel': 'linear'}
0.366 (+/-0.095) for {'C': 100, 'kernel': 'linear'}
0.366 (+/-0.095) for {'C': 1000, 'kernel': 'linear'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

           

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
